# 멜론 TOP100 크롤링


*   주소: https://www.melon.com/chart/index.htm
*   위 주소를 크롤링하여, 곡의 {곡일련번호, 순위, 앨범, 곡명, 가수, 커버이미지_주소, 좋아요}를 데이터프레임으로 정렬하여 출력하시오.



## 라이브러리 임포트 및 bs 객채 변환



In [3]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd

In [4]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [5]:
url = 'https://www.melon.com/chart/index.htm'
html = requests.get(url, headers=headers).text
html_sp = bs(html, 'html.parser')

In [ ]:
html_sp.select('span.rank')

In [ ]:
# 한 곡에 대한 test
tr_list = html_sp.select('tbody>tr')
test_tr = tr_list[0]

# 곡 일련번호
song_num = test_tr['data-song-no']
# 곡 순위
rank = test_tr.select_one('span.rank').string
# 앨범명
album_name = test_tr.select_one('.service_list_song .ellipsis.rank03 a').string
# 곡명
song_name = test_tr.select_one('.service_list_song .ellipsis.rank01 a').string
# 가수
artist = test_tr.select_one('.service_list_song .ellipsis.rank02 a').string
# 커버이미지 주소
coverimg_url = test_tr.select_one('a img')['src']
# 좋아요
# "contsLike">[0]>"SUMMCNT"
like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_num}'
like_html = requests.get(like_url, headers=headers).text
likes = int(json.loads(like_html)['contsLike'][0]['SUMMCNT'])

song_num, rank, album_name, song_name, artist, coverimg_url, likes

In [ ]:
# 모든 곡에 적용하기 위해 함수로 만들어서 실행
def get_melon_info(tr):
  # 곡 일련번호
  song_num = tr['data-song-no']
  # 곡 순위
  rank = tr.select_one('span.rank').string
  # 앨범명
  album_name = tr.select_one('.service_list_song .ellipsis.rank03 a').string
  # 곡명
  song_name = tr.select_one('.service_list_song .ellipsis.rank01 a').string
  # 가수
  artist = tr.select_one('.service_list_song .ellipsis.rank02 a').string
  # 커버이미지 주소
  coverImg_url = tr.select_one('a img')['src']
  # 좋아요
  # "contsLike">[0]>"SUMMCNT"
  like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_num}'
  like_html = requests.get(like_url, headers=headers).text
  likes = int(json.loads(like_html)['contsLike'][0]['SUMMCNT'])

  song_info = {
      '곡일련번호': [song_num],
      '순위' : [rank],
      '앨범' : [album_name],
      '곡명' : [song_name],
      '가수' : [artist],
      '커버이미지_주소' : [coverImg_url],
      '좋아요' : [likes]
  }
  song_info_df = pd.DataFrame(song_info)

  return song_info_df

In [ ]:
def get_melon_chart():
  # 함수 실행
  tr_list = html_sp.select('tbody>tr')
  melon_chart = pd.DataFrame(columns = {
      '곡일련번호' : [],
      '순위' : [],
      '앨범' : [],
      '곡명': [],
      '가수' : [],
      '커버이미지_주소' : [],
      '좋아요' : []
      })

  for tr in tr_list:
    melon_chart = pd.concat([melon_chart, get_melon_info(tr)], ignore_index = True)

  return melon_chart

In [15]:
melon_chart = get_melon_chart()

In [16]:
melon_chart.head()

,곡일련번호,순위,앨범,곡명,가수,커버이미지_주소,좋아요
0,36599950,1,NewJeans 2nd EP 'Get Up',Super Shy,NewJeans,https://cdnimg.melon.co.kr/cm2/album/images/11...,102089
1,36617841,2,Seven (feat. Latto) - Clean Ver.,Seven (feat. Latto) - Clean Ver.,정국,https://cdnimg.melon.co.kr/cm2/album/images/11...,106072
2,36635522,3,NewJeans 2nd EP 'Get Up',ETA,NewJeans,https://cdnimg.melon.co.kr/cm2/album/images/11...,42159
3,36430773,4,I feel,퀸카 (Queencard),(여자)아이들,https://cdnimg.melon.co.kr/cm2/album/images/11...,122483
4,36382580,5,1집 Alone,헤어지자 말해요,박재정,https://cdnimg.melon.co.kr/cm2/album/images/11...,88515
